In [13]:
!python -m spacy download en_core_web_sm
!spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [17]:
import pandas as pd
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, request, jsonify

# Load SpaCy model
# nlp = spacy.load('en-core-web-sm')
import en_core_web_sm
nlp = en_core_web_sm.load()
# Function to vectorize text using SpaCy
def vectorize_text(text):
    doc = nlp(text)
    return doc.vector



In [50]:
import os
import pandas
# Specify the folder path
def read_files(folder_path):
  # Initialize a list to store file names and their content
  file_data = []

  # Loop through all the files in the folder
  for filename in os.listdir(folder_path):
      if filename.endswith('.txt'):  # You can specify other file types if needed
          file_path = os.path.join(folder_path, filename)

          # Read the file content
          with open(file_path, 'r') as file:
              content = file.read()

          # Append the file name and content to the list
          file_data.append([filename, content])

  # Create a DataFrame from the list
  df = pd.DataFrame(file_data, columns=['File Name', 'Content'])
  return df

# Display the DataFrame
job_listings = read_files("jobs/")
user_profiles = read_files("users/")

In [51]:
job_listings, user_profiles

(    File Name                                            Content
 0  MLE_JD.txt  What You'll Do\n\nCreate hypotheses, design ex...
 1  BSE_JD.txt  As a Lead Backend Software Engineer in our mus...
 2  SPD_JD.txt  About The Role\n\nAs a Senior Product Designer...,
          File Name                                            Content
 0  mani_skills.txt  Jupyter Notebook\nMachine Learning\nDeep Learn...)

In [52]:
def clean_text(text):
    # Replace new lines with spaces
    text = text.replace('\n', ' ')
    # Remove extra spaces
    text = ' '.join(text.split())
    return text

# Apply the clean_text function to the 'Content' column
job_listings['Content'] = job_listings['Content'].apply(clean_text)
user_profiles['Content'] = user_profiles['Content'].apply(clean_text)

In [53]:
job_listings, user_profiles

(    File Name                                            Content
 0  MLE_JD.txt  What You'll Do Create hypotheses, design exper...
 1  BSE_JD.txt  As a Lead Backend Software Engineer in our mus...
 2  SPD_JD.txt  About The Role As a Senior Product Designer at...,
          File Name                                            Content
 0  mani_skills.txt  Jupyter Notebook Machine Learning Deep Learnin...)

In [55]:
# Step 2: Data Preprocessing
# Vectorize job descriptions and user skills
job_listings['job_vector'] = job_listings['Content'].apply(lambda x: vectorize_text(str(x)))
user_profiles['skills_vector'] = user_profiles['Content'].apply(lambda x: vectorize_text(str(x)))



In [56]:
job_listings, user_profiles

(    File Name                                            Content  \
 0  MLE_JD.txt  What You'll Do Create hypotheses, design exper...   
 1  BSE_JD.txt  As a Lead Backend Software Engineer in our mus...   
 2  SPD_JD.txt  About The Role As a Senior Product Designer at...   
 
                                           job_vector  
 0  [-0.01567027, -0.1566297, -0.1644604, 0.036854...  
 1  [-0.07880941, -0.106274664, -0.16168252, -0.08...  
 2  [-0.05281243, -0.16259919, -0.10952222, -0.008...  ,
          File Name                                            Content  \
 0  mani_skills.txt  Jupyter Notebook Machine Learning Deep Learnin...   
 
                                        skills_vector  
 0  [-0.3572886, -0.4599159, -0.11539153, 0.380577...  )

In [57]:
# Step 3: Model Training
# Create matrices from vectors
job_vectors = job_listings['job_vector'].tolist()
user_vectors = user_profiles['skills_vector'].tolist()




In [69]:
# Step 4: Generate Recommendations
recommendations = {}
for idx, row in user_profiles.iterrows():
    user_vector = user_vectors[idx].reshape(1, -1)
    cosine_similarities = cosine_similarity(user_vector, job_vectors)
    print(cosine_similarities)
    similar_jobs = cosine_similarities.argsort()[0][:-11:-1]
    recommendations[row['File Name']] = job_listings.iloc[similar_jobs]

[[0.51355183 0.48261958 0.43746725]]


In [68]:
recommendations

{'mani_skills.txt':     File Name                                            Content  \
 0  MLE_JD.txt  What You'll Do Create hypotheses, design exper...   
 1  BSE_JD.txt  As a Lead Backend Software Engineer in our mus...   
 2  SPD_JD.txt  About The Role As a Senior Product Designer at...   
 
                                           job_vector  
 0  [-0.01567027, -0.1566297, -0.1644604, 0.036854...  
 1  [-0.07880941, -0.106274664, -0.16168252, -0.08...  
 2  [-0.05281243, -0.16259919, -0.10952222, -0.008...  }